In [1]:
# ENVIRONMENT

from aisurveywriter.tasks import ReferenceExtractor
from aisurveywriter.core.llm_handler import LLMHandler
import aisurveywriter.core.file_handler as fh
from aisurveywriter.utils import get_all_files_from_paths
from aisurveywriter.core.pipeline import PaperPipeline
from aisurveywriter.core.paper import PaperData
import aisurveywriter.tasks as tks

import os
os.environ["GOOGLE_API_KEY"]=fh.read_credentials("../credentials.yaml")["google_key"]

# llm = LLMHandler(model="qwen2.5:14b", model_type="ollama", temperature=0.5)
prompts = fh.read_yaml("../templates/prompt_config.yaml")
review = fh.read_yaml("../templates/review_config.yaml")


# ref = tks.ReferenceExtractor(llm, ref_paths=["../refexamples/ArigaK2023_ChemOfMat.pdf"],
#                              prompt=prompts["reference_extract_prompt"],
#                              raw_save_path="raw.ref", rawbib_save_path="raw.bib",
#                              bib_save_path="final.bib", cooldown_sec=0)
# ref()

In [ ]:
from aisurveywriter.tasks.paper_faiss_ref import PaperFAISSReferencer, PaperData
from aisurveywriter.core.text_embedding import load_embeddings
from aisurveywriter.core.latex_handler import write_latex

paper = PaperData.from_tex("../out/generated.tex", subject="Langmuir and Langmuir-Blodgett films")
paper.sections = paper.sections[1:]

embed = load_embeddings("Alibaba-NLP/gte-Qwen2-1.5B-instruct", "huggingface")
ref = PaperFAISSReferencer(embed, "../bib/refextract-21papers.bib", local_faissdb_path="../bib/qwen2-1.5b", save_usedbib_path="temp/test.bib", 
                           save_faiss_path="temp/qwen2-1.5b", max_per_section=40, max_per_sentence=1,confidence=0.9)

paper = ref.reference(paper)
write_latex("../templates/paper_template.tex", paper, "temp/test.tex", bib_path="temp/test.bib")

In [ ]:
from aisurveywriter.tasks.figure_extractor import FigureExtractor, PaperData
from aisurveywriter.core.llm_handler import LLMHandler
from aisurveywriter.core.text_embedding import load_embeddings
from aisurveywriter.core.latex_handler import write_latex
from aisurveywriter.utils import get_all_files_from_paths

llm = LLMHandler(model="gemini-2.0-flash", model_type="google")
embed = load_embeddings(model="Alibaba-NLP/gte-Qwen2-1.5B-instruct", model_type="huggingface")
paper = PaperData.from_tex("../out/generated.tex")
fig = FigureExtractor(llm, embed, paper.subject, pdf_paths=get_all_files_from_paths("../refexamples"), 
                      save_dir="./temp", faiss_save_path="./temp/mpnetbasev2-imgfaiss", local_faiss_path="./temp/mpnetbasev2-imgfaiss", paper=paper, request_cooldown_sec=20)

paper = fig(paper)
write_latex("../templates/paper_template.tex", paper)

In [3]:
write_latex("../templates/paper_template.tex", paper, "test.tex")